---
# Imports

In [6]:
import pyspark 
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [7]:
spark = SparkSession.builder.appName("ProjetoABD").getOrCreate()

In [8]:
# Download da versão original do dataset
import kagglehub
path = kagglehub.dataset_download("robikscube/flight-delay-dataset-20182022")
print("Path to dataset files:", path)

Path to dataset files: /home/jovyan/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4


In [9]:
path='/home/jovyan/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4'
mydata = spark.read.load(path,format='csv',sep=',',inferSchema=True, header=True)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
mydata.printSchema()
mydata.show()
mydata.count()

root
 |-- FlightDate: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelayMinutes: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelayMinutes: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |-- DOT

67761365

-----
# Data Cleaning

## 1.Removing duplicates if any

In [ ]:
# Checking first: use dropDuplicates() and count()
print(f'  Dados: number of rows: {mydata.count() }, after dropduplicates: {mydata.dropDuplicates().count() }')

In [ ]:
#Remover duplicados e guardar num novo csv
mydata = mydata.dropDuplicates()
mydata.write.option("header", "true") .csv("flights.csv")

In [ ]:
#Leitura do novo csv sem duplicados, apenas 50000 registos
data_clean = spark.read.load('flights.csv',format='csv',sep=',',inferSchema=True, header=True).limit(50000)

## 2.Handling missing values

In [ ]:
# Ver em que colunas estão os nulos (by columns)
dict_nulls_calls = {col: data_clean.filter(data_clean[col].isNull()).count() for col in data_clean.columns}
dict_nulls_calls

In [ ]:
#Visto que todas as colunas tem alguns nulls, vale a pena eliminar colunas ou retirar as linhas?
cols_to_dismiss = []
data_calls = data_clean.drop(*cols_to_dismiss)

In [ ]:
#Quantas linhas tem valores nulos
#print(f"After dropna would be: data_iot = {data_clean.dropna(how='any').count()  } rows")
data_clean=data_clean.dropna(how='any')
data_clean.count()


1967

In [11]:
# Show one record in vertical mode for better reading
data_clean.show(1,vertical=True)


-RECORD 0---------------------------------------------------
 FlightDate                              | 2020-09-20       
 Airline                                 | Comair Inc.      
 Origin                                  | CLT              
 Dest                                    | JAX              
 Cancelled                               | False            
 Diverted                                | False            
 CRSDepTime                              | 1314             
 DepTime                                 | 1309.0           
 DepDelayMinutes                         | 0.0              
 DepDelay                                | -5.0             
 ArrTime                                 | 1439.0           
 ArrDelayMinutes                         | 0.0              
 AirTime                                 | 61.0             
 CRSElapsedTime                          | 86.0             
 ActualElapsedTime                       | 90.0             
 Distance               

-----
# Data transformation